<a href="https://colab.research.google.com/github/alfitrinurqalbi-blip/5th-week-material-of-Spatial-Programming/blob/main/YOLO_Sample_BuildingSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Persiapan Dependensi dan Data

Cek GPU

In [2]:
!nvidia-smi

Thu Nov 20 21:38:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Install dependensi Ultralytics, Custom utility dan Roboflow

In [1]:
!git clone https://github.com/fakmalpradana/YOLO-BuildingSegmentation-Simple.git
!pip install ultralytics roboflow patchify
!cp -r '/content/YOLO-BuildingSegmentation-Simple/utils/fakmalpradana-yolo-utils' '/content/utils'
!cp '/content/YOLO-BuildingSegmentation-Simple/yolo11m-seg.pt' '/content/yolo11m-seg.pt'
!cp -r '/content/YOLO-BuildingSegmentation-Simple/orthomosaic' '/content/orthomosaic'
!cp '/content/YOLO-BuildingSegmentation-Simple/Sample-YOLO.v1-sample-training-01.yolov11.zip' '/content/Sample-YOLO.v1-sample-training-01.yolov11.zip'
!cp '/content/YOLO-BuildingSegmentation-Simple/bontang-model.pt' '/content/bontang-model.pt'
!rm -rf YOLO-BuildingSegmentation-Simple
!mkdir public
!mkdir public/patch-temp
!mkdir output

Cloning into 'YOLO-BuildingSegmentation-Simple'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 55 (delta 7), reused 13 (delta 3), pack-reused 32 (from 3)
Receiving objects: 100% (55/55), 189.97 MiB | 27.10 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (40/40), done.
cp: cannot stat '/content/YOLO-BuildingSegmentation-Simple/bontang-model.pt': No such file or directory
mkdir: cannot create directory ‘public’: File exists
mkdir: cannot create directory ‘public/patch-temp’: File exists
mkdir: cannot create directory ‘output’: File exists


In [2]:
import zipfile

# Define the path to your ZIP file and the extraction directory
zip_path = '/content/Sample-YOLO.v1-sample-training-01.yolov11.zip'
extract_path = '/content/dataset/'  # or any preferred location

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


## Training

In [3]:
from ultralytics import YOLO


model = YOLO('/content/yolo11m-seg.pt')


results = model.train(
    data='/content/dataset/data.yaml',
    epochs=100,
    imgsz=640,
    name='model-bontang',
    batch=4,
    patience=0,
    mask_ratio=1,
    cls=1.0,
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=1.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=Fal

## Predict

In [4]:
import os
import cv2
import time
import numpy as np

from tqdm.auto import tqdm
from glob import glob
from utils.json_ import jsonYOLO, Converter
from utils.segment import Predict, delete_patch_temp
from utils.tile import tile_img
from utils.raster_metadata import meta
from google.colab import files

In [5]:
start = time.time()

# INPUT ADA DISINI --EDIT DI SINI SINI AJA--

# MODEL = 'public/model/3-besar/best.pt'
MODEL = '/content/bontang-model.pt'
# image path
IMG = '/content/orthomosaic/sample-bontang-1.tif'
# konstanta mengatur overlap || nilai overlap = 1 - nilai yang di input
OVER = 0.97
# mengatur tipe output || masks atau boxes
TIPE = 'masks'




# BAGIAN PROSESING --JANGAN DIEDIT--

# tile data image
img, _ = tile_img(IMG, DIM=640).patchData(patch_dim=640, step_size=OVER)
print(f'shape tile dataset : {img.shape}')

# write tile data temporary
for i in range(img.shape[0]):
    cv2.imwrite(f'public/patch-temp/patch{i+10001}.png', img[i])

# Line dibawah untuk batch predict
LIST_IMG = glob('public/patch-temp/patch*.png')
with tqdm(total=len(LIST_IMG), desc='Proses deteksi bangunan', leave=True) as pbar:
    for i in LIST_IMG:
        a = Predict(img=i, model=MODEL).to_json(i.replace('png','json'), TIPE)
        pbar.update(1)

# Line untuk combine json
print('combine seluruh hasil prediction ...')
Predict.combine_json(OVER, TIPE)

# Convert json ke geojson
inp = 'public/patch-temp/combined_file.json'
out = '/content/output.geojson'

ext = meta(IMG).cari_extent()
epsg = meta(IMG).get_epsg_from_raster()

Converter(inp, out, epsg=epsg, extent=ext, dim=_, DIM=640).to_geojson()
files.download(out)

end = time.time()
mnt = (end - start)//60
dtk = np.ceil((end - start)- (mnt * 60))
print(f'Output berhasil di download')
print(f'--Alhamdulillah selesai dalam {mnt} menit {dtk} detik--')


(4480, 4480, 3)
shape tile dataset : (49, 640, 640, 3)


Proses deteksi bangunan:   0%|          | 0/49 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/bontang-model.pt'